In [ ]:
https://github.com/DirkEngfer/PandasTests
Hourly NO2 measures in Bornhoeved (Schleswig-Holstein/Germany). programmer: Dirk Engfer, Germany
Provided Input data file: NO2_2019_Bornhoeved.csv

------------------------------
Original source of Input Data:
------------------------------
Quelle: Umweltbundesamt, https://www.umweltbundesamt.de/daten/luft/luftdaten/stationen
        (Abruf: 02.10.2020). Alle Uhrzeiten sind in der jeweils zum Messzeitpunkt
        gültigen Zeit (MEZ bzw. MESZ) angegeben.


In [1]:
import os, numpy as np
import pandas as pd
homedir = os.getenv('HOME')

datapath = os.path.join(homedir, 'Dokumente','python-apps','tensorflow', 'eu_air_pollution_data')
datafile = 'NO2_2019_Bornhoeved.csv'

indatapath = os.path.join(datapath,datafile)

In [ ]:
Translate the SAS SET statement and friends into Pandas:

In [ ]:
df = pd.read_csv(indatapath, header=0, sep=',',usecols=[2,7,8,9])
df = df.loc[(df.Stationsname.isin(['Bornhöved', 'Kiel-Bahnhofstr. Verk.']))]
df = df.loc[(df.Messwert.str.contains(pat='-')) == False]
df['Messwert']  = df['Messwert'].astype(np.int)
from datetime import datetime
f = lambda x:datetime.strptime(x[-10:], "%d.%m.%Y")
df['mydate'] = df['Datum'].map(f)

df.sort_values(by=['Stationsname', 'mydate', 'Messwert'], ascending=[True,True, False], inplace=True, axis=0)
dailymean = df.groupby(['Stationsname', 'mydate'], sort=False, as_index=False).nth([0,1,2]).groupby(['Stationsname', 'mydate'], sort=False, as_index=False).mean()
dailymean['day_count'] = dailymean.groupby('Stationsname', sort=False, as_index=False).cumcount().add(1)
dailymean['Jahresgrenzwert'] = 40
Ki = dailymean.loc[(dailymean.Stationsname.isin(['Kiel-Bahnhofstr. Verk.']))]
Bo = dailymean.loc[(dailymean.Stationsname.isin(['Bornhöved']))]
Bo2 = Bo.copy()
Bo2['Bornh'] = 'yes'

In [ ]:
Set together 2 dataframes,
  (1) perform Append
  (2) perform Concatenation
  (3) Compare methods 1 and 2 for differences in the resulting dataframe

In [18]:
_appended = Ki.append(Bo2, ignore_index=True, verify_integrity=False, sort=False)
_concatenated = pd.concat([Ki, Bo2], join='outer', axis=0)
compared = _appended.merge(_concatenated, on=['Stationsname', 'mydate', 'Messwert', 'day_count', 'Jahresgrenzwert', 'Bornh'], how='inner', indicator=True)
print(compared.loc[compared._merge != 'both', :])

# Some more tests on Merge facets:
d1 = pd.DataFrame({'x':['x1', 'x2', 'x2', 'x3'], 'y':['y1', 'y2a','y2b', 'y3'], 'z': ['z1', 'z2a','z2b', 'z3']})
d1
d2 = pd.DataFrame({'x':['x2', 'x3', 'x40', 'x50'], 'y':['y10', 'y20','y20', 'y30'], 'z': ['z10', 'z20','z20', 'z30']})
d3 = d1.merge(d2, on='x', how='outer')
d4 = d1.merge(d2, on='x', how='inner')


Empty DataFrame
Columns: [Stationsname, mydate, Messwert, day_count, Jahresgrenzwert, Bornh, _merge]
Index: []
